### Packages

In [ ]:
## packages
import math
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.initializers import GlorotUniform, GlorotNormal, HeUniform, HeNormal, Zeros, Ones
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model
from scipy.optimize import minimize
from scipy.stats import linregress
from joblib import Parallel, delayed
import psutil

### Real-Time Data Processing

In [ ]:
## set of function to read and process the raw spectra
def read_files(loc):
    files_all = sorted(os.listdir(loc), key = len)
    ABS_names = []
    PL_names = []
    ABS_all = []
    PL_all = []
    for file in files_all:
        if (file.startswith("Abs")):
            ABS = pd.read_csv(loc + "//" + file , header = None).to_numpy()
            ABS_all.append(ABS)
            ABS_names.append(file.split(".")[0])
        elif (file.startswith("PL")):
            PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
            PL_all.append(PL)
            PL_names.append(file.split(".")[0])
        elif(file.startswith("Wavelength_Abs")):
            WL_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("Wavelength_PL")):
            WL_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_Abs")):
            DR_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_PL")):
            DR_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("LR")):
            LR = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("FR")):
            FR = pd.read_csv(loc + "//" + file, header = None).to_numpy()    
    return WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR
def idx_min(y, x):
    diff = np.abs(y[:, 0] - x)
    idx = diff.argmin()
    return idx
# extract reactive phase
def extract(x, idx1, idx2, numFirstElements):
    x_mean = x[idx1:idx2 + 1, :].mean(axis = 0)
    x_sort = np.sort(x_mean)
    idx_phase = np.nonzero(np.in1d(x_mean, x_sort[:numFirstElements]))[0]
    x_phase = x[:, idx_phase]
    return x_phase
def spectra_avg(x):
    x_avg = x.mean(axis = 1) # avg over extracted reactive phase spectra for each WL; will result in a row matrix
    x_avg = x_avg[:, np.newaxis]
    return x_avg
def baseline_zero(x, idx_low, idx_high):
    x_baseline_zero = x - x[idx_low:idx_high + 1, :].mean() # make baseline zero; subtracting mean of PL at LL - HL nm
    return x_baseline_zero
def linear_int_x(y, x, i, y_btw):
    x_btw = x[i, 0] - ((x[i + 1, 0] - x[i, 0]) / (y[i + 1, 0] - y[i, 0])) * (y[i, 0] - y_btw)
    return x_btw
def linear_int_y(y, x, i, x_btw):
    y_btw = y[i, 0] - ((y[i + 1, 0] - y[i, 0]) / (x[i + 1, 0] - x[i, 0])) * (x[i, 0] - x_btw)
    return y_btw
def peak_info(y, x, emission_intensity):
    # correction for emission peak intensity in case min is not exactly zero
    # min_intensity = y.min()
    min_intensity = 0 # without correction
    intensity_peak = emission_intensity - min_intensity
    # emission peak area
    area_peak = np.trapz(y.T, x = x.T)[0]
    # emission peak intensity and area
    output = [intensity_peak, area_peak]
    return np.array(output)[:, np.newaxis]
# process ABS
def spectra_extract_ABS(WL, DR, LR, ABS_all, ABS_names):
    # baseline WL limits
    baseline_LL = 700
    baseline_HL = 800
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    lastPeak_LL = 250
    lastPeak_HL = 350 
    idx_lastPeak_LL = idx_min(WL, lastPeak_LL)
    idx_lastPeak_HL = idx_min(WL, lastPeak_HL)
    # excitation WL info
    excitation_WL = 300
    idx_excitationWL = idx_min(WL, excitation_WL)
    # initiate files to be exported
    WLABS_processed = WL
    ABS_excitationWL_list = []
    # ABS_excitationWL_list = np.array([["Absorbance at excitation WL"]])
    for (item, name) in zip(ABS_all, ABS_names):
        ABS_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        ABS_phase = extract(ABS_only, idx_lastPeak_LL, idx_lastPeak_HL, param)
        ABS_phase_avg = spectra_avg(ABS_phase) 
        arg_log = (ABS_phase_avg - DR) / (LR - DR) # Beer-Lambert law
        ABS_processed = - np.log10(arg_log) # Beer-Lambert law
        ABS_processed_baseline = baseline_zero(ABS_processed, idx_WL_LL, idx_WL_HL) 
        WLABS_processed = np.concatenate( [WLABS_processed, ABS_processed_baseline] , axis = 1) # attach WL and processed ABS
        # WLABS_processed_filtered = WLABS_processed[np.isnan(WLABS_processed[:, 1]) == False] # remove NaN ABS
        ABS_excitationWL = linear_int_y(ABS_processed_baseline, WL, idx_excitationWL, excitation_WL)
        ABS_excitationWL_list.append(ABS_excitationWL)
        # ABS_excitationWL_list = np.concatenate( [ABS_excitationWL_list, np.array([ABS_excitationWL])[:, np.newaxis]] , axis = 1)
    ABS_excitationWL_list = np.array(ABS_excitationWL_list)[:, np.newaxis]
    return ABS_excitationWL_list
# process PL
def spectra_extract_PL(WL, DR, PL_all, PL_names):
    ## baseline WL limits
    baseline_LL = 800
    baseline_HL = 1000
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    excWL_LL = 250
    excWL_HL = 350
    idx_excWL_LL = idx_min(WL, excWL_LL)
    idx_excWL_HL = idx_min(WL, excWL_HL)
    ## emission peak WL and the WL range for the right side of emission peak
    emPeak_WL = 446
    emPeak_LL = 446
    emPeak_HL = 700
    idx_emPeak_LL = idx_min(WL, emPeak_LL)
    idx_emPeak_HL = idx_min(WL, emPeak_HL)
    WL_aroundPeak = WL[idx_emPeak_LL: idx_emPeak_HL + 1, :]
    idx_intensityWL = idx_min(WL, emPeak_WL) # index for emission peak WL
    ## initiate files to be exported
    WLPL_processed = WL
    emission_details_list = []
    # emission_details = np.array([["Emission Peak WL"], ["Emission Peak Intensity"], ["FWHM"], ["Emission Peak Area"]])
    for (item, name) in zip(PL_all, PL_names):
        PL_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        PL_phase = extract(PL_only, idx_excWL_LL, idx_excWL_HL, param) 
        PL_phase_avg = spectra_avg(PL_phase)
        PL_processed = PL_phase_avg - DR
        PL_processed_baseline = baseline_zero(PL_processed, idx_WL_LL, idx_WL_HL) 
        WLPL_processed = np.concatenate( [WLPL_processed, PL_processed_baseline] , axis = 1) # attach WL and processed PL
        ## emission peak info
        PL_emissionWL = linear_int_y(PL_processed_baseline, WL, idx_intensityWL, emPeak_WL)
        PL_aroundPeak = PL_processed_baseline[idx_emPeak_LL: idx_emPeak_HL + 1, :]
        info_peak = peak_info(PL_aroundPeak, WL_aroundPeak, PL_emissionWL)
        emission_details_list.append(info_peak)
        # emission_details = np.concatenate( [emission_details, info_peak_method] , axis = 1)
    emission_details_list = np.concatenate(emission_details_list, axis = 1).T
    emission_details_list[:,[1]] = emission_details_list[:,[1]] * 2 # since we calculated the area under the curve for the half right side of the emission peak
    return emission_details_list[:, [0]] # emission peak intensity as the single output of interest

### ML Modeling

In [ ]:
## scale output
def non_dim_y(y_initial):
    y_scaled = y_initial / 5000
    return y_scaled
## nondimensionalize inputs
def non_dim_x(x_initial):
    x_normalized = np.zeros((x_initial.shape[0], x_initial.shape[1] - 4))
    x_normalized[:, 0] = (x_initial[:, 0] - 120) / (150 - 120) # x1 = temperature
    x_normalized[:, 1] = (x_initial[:, 1] - 40) / (90 - 40) # x2 = CuI
    x_normalized[:, 2] = (x_initial[:, 2] - 15) / (30 - 15) # x3 = OA-CuI
    x_normalized[:, 3] = (x_initial[:, 3] - 15) / (30 - 15) # x4 =  OAm-CuI
    x_normalized[:, 4] = (x_initial[:, 4] - 80) / (100 - 80) # x5 = ODE-CuI
    x_normalized[:, 5] = (x_initial[:, 5] - 40) / (90 - 40) # x6 = CsOleate
    x_normalized[:, 6] = (x_initial[:, 6] - 15) / (30 - 15) # x7 = OA-CsOleate
    return x_normalized
## split dataset to training and validation
def split_set(x, y, train_ratio = 0.8, seed_num = 100):
    xy_all = np.concatenate((x, y), axis = 1)
    np.random.seed(seed_num)  
    np.random.shuffle(xy_all)
    # training set
    xy_train = xy_all[:int(train_ratio * xy_all.shape[0]), :]
    x_train = xy_train[:, :-y.shape[1]]
    y_train = xy_train[:, -y.shape[1]:]
    # validation set
    xy_test = xy_all[int(train_ratio * xy_all.shape[0]):, :]
    x_test = xy_test[:, :-y.shape[1]]
    y_test = xy_test[:, -y.shape[1]:]
    return x_train, y_train, x_test, y_test
## ENN structure
def structure_info(num_models, layer_min = 6, layer_max = 8, node_min = 15, node_max = 30, seed_num = 100):
    random.seed(seed_num)
    detail_all = []
    for i in range (num_models):
        num_layers = random.randint(layer_min, layer_max)
        nodes_list = []
        for j in range(num_layers):
            num_nodes = random.randint(node_min, node_max)
            nodes_list.append(num_nodes)
        detail = [num_layers, nodes_list]
        detail_all.append(detail)
    return detail_all
## build and train an ensemble of cascade NNs
def parallel_train_ML(train_x, train_y, val_x, val_y, structure):
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    ## cascade NN
    input = Input(shape = train_x.shape[1])
    x = input
    for layer in range(structure[0]):
        output = Dense(structure[1][layer], activation = 'relu', kernel_initializer = HeUniform(seed = 42), bias_initializer = Zeros())(x)
        x = Concatenate()([x, output])
    # dropout = layers.Dropout(0.1)(x)
    # output = Dense(train_y.shape[1], activation = 'linear')(dropout)
    output = Dense(train_y.shape[1], activation = 'linear')(x)
    model = Model(inputs = input, outputs = output)
    ## train the model
    # early stopping terminates training if validation model loss begins to increase due to over-fitting
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 50, min_delta = 1e-5, mode = 'min', restore_best_weights = True)
    # early_stopping = EarlyStopping(monitor = 'val_loss', patience = 50, min_delta = 1e-5, mode = 'min')
    # model compile with loss, optimizer, and performance metric of interest
    model.compile(loss = 'mean_squared_error', optimizer = RMSprop(learning_rate = 1e-4), metrics = [RootMeanSquaredError()])
    # perform Model Training with training and validation data with early stopping
    history = model.fit(train_x, train_y, validation_data = (val_x, val_y), epochs = 1000, verbose = 0, callbacks = [early_stopping])
    result = [model, history]
    return result
## deconvolute the model and training history
def deconvolute_ML(ensemble_result):
    ensemble_model = []
    ensemble_history = []
    for element in ensemble_result:
        ensemble_model.append(element[0])
        ensemble_history.append(element[1])
    return ensemble_model, ensemble_history
## plots related to the training process
def learn_plot(history):
    fig = plt.figure()
    # loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], color = 'blue')
    plt.plot(history.history['val_loss'], color = 'red')
    plt.title('Loss (MSE) - Epoch')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    # accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['root_mean_squared_error'], color = 'blue')
    plt.plot(history.history['val_root_mean_squared_error'], color = 'red')
    plt.title('RMSE - Epoch')
    plt.ylabel('RMSE')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    fig.tight_layout()
    plt.show()
## plot related to the predicted vs. true outputs - for individual models    
def predict_plot_separate(model, x, y_true):
    y_true = np.squeeze(y_true)
    predicted_y = model.predict(x, verbose = 0)
    detail_1 = linregress(y_true, np.squeeze(predicted_y))
    # detail_2 = linregress(y_true[:, 1], predicted_y[:, 1])
    plt.scatter(y_true, np.squeeze(predicted_y), color = 'blue')
    # plt.scatter(y_true[:, 1], predicted_y[:, 1], color = 'red')
    plt.plot([0, 1],[0, 1], color = 'green')
    plt.ylabel('Predicted')
    plt.xlabel('Actual')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    # plt.legend(['y0: $R^2 = $' + str(detail_1.rvalue ** 2), 'y1: $R^2 = $' + str(detail_2.rvalue ** 2), 'Parity'])
    plt.legend(['y0: $R^2 = $' + str(detail_1.rvalue ** 2), 'Parity'])
    plt.show()
## plot related to the predicted vs. true outputs - for the entire ENN  
def predict_plot_mean(ensemble_NN, x, y_true):
    y_true = np.squeeze(y_true)
    y_pred = []
    for model in ensemble_NN:
        y_pred.append(model.predict(x, verbose = 0))
    y_pred_arr = np.concatenate(y_pred, axis = 1)
    out_mean = np.mean(y_pred_arr, axis = 1)
    out_std = np.std(y_pred_arr, axis = 1)
    out_detail = linregress(y_true, out_mean)
    plt.errorbar(y_true, out_mean, yerr = out_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = 'y: $R^2 = $' + str(round((out_detail.rvalue ** 2), 4)))
    plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
    plt.ylabel('Predicted')
    plt.xlabel('Actual')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.legend()
    return out_mean, out_std
## save the ENN in the given path
def save_ENN(ENN, path):
    for n, model in enumerate(ENN):
        os.mkdir(path + "//" + str(n))
        model.save(path + "//" + str(n))
## load the ENN stored in the given path
def load_plot(path):
    model_idx = sorted(os.listdir(path), key = len)
    ensemble = []
    for idx in model_idx:
        ensemble.append(load_model(path + "//" + idx))
    return ensemble

### Optimization

In [ ]:
# used of expected improvement (EI)
def best_target_data(y_norm, y_target):
    obj_fun_data = abs(y_norm - y_target)
    obj_fun_best = obj_fun_data.min()
    return obj_fun_best
## aquisition functions implementation
def apply_policy(x_initial, ensemble, y_target, obj_fun_best, policy):
    # p = psutil.Process()
    # p.nice(psutil.REALTIME_PRIORITY_CLASS)    
    x = x_initial[np.newaxis, :]
    obj_models = []
    for model in ensemble:
        y_pred = model.predict(x, verbose = 0)
        # our ptoblem is single objective optimization
        obj_fun = abs(y_target - np.squeeze(y_pred)[()])
        obj_models.append(obj_fun)
    ## EI
    if (policy[0] == "EI"):
        imp_list = []
        for element in obj_models:
            diff = obj_fun_best - element 
            if (diff >= 0):
                imp_list.append(diff)
            else:
                imp_list.append(0)
        value = - np.mean(imp_list)
    ## UCB
    if (policy[0] == "UCB"):
        value = np.mean(obj_models) - (policy[1] * np.std(obj_models))
    ## MV      
    if (policy[0] == "MV"):
        value = - np.std(obj_models)
    ## EPLT
    elif (policy[0] == "EPLT"):
        value =  np.mean(obj_models)
    return value
## optimization
def opt_general(f, dim):
    # p = psutil.Process()
    # p.nice(psutil.REALTIME_PRIORITY_CLASS)
    low_limit = [0] * dim
    up_limit = [1] * dim
    bnds = list(zip(low_limit, up_limit))
    count_success = 0
    while (count_success < 1):
        x_initial = np.random.uniform(0, 1, dim)
        # sol = minimize(f, x_initial, method = "trust-constr", bounds = bnds, options = {'gtol': 0.02, 'xtol': 0.02, 'verbose': 0})
        sol = minimize(f, x_initial, method = "SLSQP", bounds = bnds, options = {'ftol': 0.02, 'maxiter': 200})
        if ((sol.success == True) and (sol.nit > 1)):
            result = sol
            count_success = count_success + 1
    return result      
def parallel_opt(ensemble, y_target, obj_fun_best, policy, x_shape):
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    model_pol = lambda x: apply_policy(x, ensemble, y_target, obj_fun_best, policy)
    solution = opt_general(model_pol, x_shape)
    return solution
## use this while running multiple optimization; here we used this for MV explorative campaign
# def deconvolute_opt(solution, dim, num_restart, num_selection, threshold_ratio = 0.3):
#     xy = np.zeros((num_restart, dim + 1))
#     for count in range(len(solution)):
#         xy[count, :-1] = solution[count].x
#         xy[count, -1] = solution[count].fun
#     xy_sort = xy[xy[:, -1].argsort()]
#     dist_arr = np.linalg.norm(xy_sort[1:, :-1] - xy_sort[[0], :-1], axis = 1)
#     threshold = threshold_ratio * np.amax(dist_arr)
#     final_xy = xy_sort[[0], :]
#     for i in range(1, xy_sort.shape[0]):
#         if np.all(np.linalg.norm(final_xy[:, :-1] - xy_sort[[i], :-1], axis = 1) > threshold):
#             final_xy = np.vstack((final_xy, xy_sort[[i], :]))
#     select_xy = final_xy[:num_selection, :]
#     return select_xy
## use this while running one single optimization; here we used this for EI, UCB, and EPLT optimization campaigns 
def deconvolute_opt(solution, dim, num_restart, num_selection):
    xy = np.zeros((num_restart, dim + 1))
    for count in range(len(solution)):
        xy[count, :-1] = solution[count].x
        xy[count, -1] = solution[count].fun
    select_xy = xy[:num_selection, :]
    return select_xy
## use this while running multiple optimization; here we used this for MV explorative campaign  
# def exp_sel(y, FR, y_target, policy, num_selection, num_models = 20, num_restart = 5):
#     ## dataset and preprocessing needed to be able to train the model
#     y_norm = non_dim_y(y)
#     obj_fun_best = best_target_data(y_norm, y_target)
#     # a = y.shape[0]
#     # x_norm = non_dim_x(FR[:a, :])
#     x_norm = non_dim_x(FR)
#     x_train, y_train, x_valid, y_valid = split_set(x_norm, y_norm)
#     ## train the ENN
#     start_ML = time()
# #     p = psutil.Process()
# #     p.nice(psutil.REALTIME_PRIORITY_CLASS)
#     arthitecture = structure_info(num_models)
#     result_aggregate_ML = Parallel(n_jobs = -2)(delayed(parallel_train_ML)(x_train, y_train, x_valid, y_valid, arthitecture[i]) for i in range(num_models))
#     ensemble, history = deconvolute_ML(result_aggregate_ML)
#     end_ML = time()
#     print("------------------------------------------------------------------------------------------------------------")
#     print(f"The execution time to train the ensemble NN is {str(end_ML - start_ML)} seconds.")
#     print("------------------------------------------------------------------------------------------------------------")
#     ## optimization and achive the best input variables needed for the future experiments
#     start_opt = time()
# #     p = psutil.Process()
# #     p.nice(psutil.REALTIME_PRIORITY_CLASS)
#     result_aggregate_opt = Parallel(n_jobs = -2)(delayed(parallel_opt)(ensemble, y_target, obj_fun_best, policy, x_norm.shape[1]) for i in range(num_restart))
#     xy_best = deconvolute_opt(result_aggregate_opt, x_norm.shape[1], num_restart, num_selection)
#     # sort based on the temperature
#     xy_best_sort = xy_best[xy_best[:, 0].argsort()]
#     x_best = xy_best_sort[:, :-1]
#     y_best = xy_best_sort[:, [-1]]
#     end_opt = time()
#     print("------------------------------------------------------------------------------------------------------------")
#     print(f"The execution time for optimization is {str(end_opt - start_opt)} seconds.")
#     print("------------------------------------------------------------------------------------------------------------")
#     return x_best, y_best, result_aggregate_opt, ensemble, history, x_train, y_train, x_valid, y_valid, arthitecture
## use this while running one single optimization; here we used this for EI, UCB, and EPLT optimization campaigns
def exp_sel(y, FR, y_target, policy, num_selection, num_models = 20, num_restart = 1):
    ## dataset and preprocessing needed to be able to train the model
    y_norm = non_dim_y(y)
    obj_fun_best = best_target_data(y_norm, y_target)
    # a = y.shape[0]
    # x_norm = non_dim_x(FR[:a, :])
    x_norm = non_dim_x(FR)
    x_train, y_train, x_valid, y_valid = split_set(x_norm, y_norm)
    ## train the ENN
    start_ML = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    arthitecture = structure_info(num_models)
    result_aggregate_ML = Parallel(n_jobs = -2)(delayed(parallel_train_ML)(x_train, y_train, x_valid, y_valid, arthitecture[i]) for i in range(num_models))
    ensemble, history = deconvolute_ML(result_aggregate_ML)
    end_ML = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time to train the ensemble NN is {str(end_ML - start_ML)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    ## optimization and achive the best input variables needed for the future experiments
    start_opt = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    result_aggregate_opt = Parallel(n_jobs = -2)(delayed(parallel_opt)(ensemble, y_target, obj_fun_best, policy, x_norm.shape[1]) for i in range(num_restart))
    xy_best = deconvolute_opt(result_aggregate_opt, x_norm.shape[1], num_restart, num_selection)
    # sort based on the temperature
    x_best = xy_best[:, :-1]
    y_best = xy_best[:, [-1]]
    end_opt = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time for optimization is {str(end_opt - start_opt)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    return x_best, y_best, result_aggregate_opt, ensemble, history, x_train, y_train, x_valid, y_valid, arthitecture

### Master Code

In [ ]:
def count_files(path):
    files_all = os.listdir(path)
    count_ABS = 0
    count_PL = 0
    for file in files_all:
        if (file.startswith("Abs")):
            count_ABS = count_ABS + 1
        elif (file.startswith("PL")):
            count_PL = count_PL + 1
    return count_ABS, count_PL
def x_dim(x_norm, FR, path):
    x_final = np.zeros((x_norm.shape[0], x_norm.shape[1] + 4))
    x_final[:, 0] = (x_norm[:, 0] * (150 - 120)) + 120
    x_final[:, 1] = (x_norm[:, 1] * (90 - 40)) + 40
    x_final[:, 2] = (x_norm[:, 2] * (30 - 15)) + 15
    x_final[:, 3] = (x_norm[:, 3] * (30 - 15)) + 15
    x_final[:, 4] = (x_norm[:, 4] * (100 - 80)) + 80
    x_final[:, 5] = (x_norm[:, 5] * (90 - 40)) + 40
    x_final[:, 6] = (x_norm[:, 6] * (30 - 15)) + 15
    # last four columns
    x_final[:, 7] = np.sum(x_final[:, 1:5], axis = 1) - np.sum(x_final[:, 5:7], axis = 1) # x8 = ODE-CsOleate
    x_final[:, -3] = np.sum(x_final[:, 1:8], axis = 1) # x9 = PFO
    x_final[:, -2] = 2 # x10 = 2 always; wait for 2 residence times then record the spectra
    # x_final[:, -1] = 0 # x11 = 0 always; no washing
    updated_FR = np.concatenate((FR, x_final), axis = 0) ## concat previous and new input parameters
    np.savetxt(path + "//" + "FR.csv", updated_FR, delimiter = ",", fmt="%1.3f") # overwrite it into the existing FR.csv file
    return x_final, updated_FR
## master function
## use this while running multiple optimization and selecting 2 with an algorithm for 1 in que; here we used this for MV explorative campaign 
# def master_code(y_target = 1.0, policy = ["MV", 0], num_selection = 2, exp_budget = 61): # to test using the provided dataset
# def master_code(y_target = 1.0, policy = ["MV", 0], num_selection = 2, exp_budget = 59):
#     loc = input("Please enter the path for the directory that includes files: ")
#     print("------------------------------------------------------------------------------------------------------------")
#     num_files_old = 0
#     while True:
#         num_ABS, num_PL = count_files(loc)
#         if (num_ABS == num_PL):
#             num_files = num_PL
#         if (num_files > num_files_old) and (num_files < exp_budget):
#             num_files_old = num_files
#             WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR = read_files(loc)
#             if (num_files > FR.shape[0] - 2):
#                 print("New run starts now:")
#                 y = spectra_extract_PL(WL_PL, DR_PL, PL_all, PL_names)
#                 FR_used = FR[:num_files, :]
#                 new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure = exp_sel(y, FR_used, y_target, policy, num_selection)
#                 new_FR, updated_FR = x_dim(new_x, FR, loc)
#         elif (num_files >= exp_budget):
#             print("The experimental budget is reached!")
#             break
#         ## use this for the test run using the provided dataset 
#         # if (num_files_old == 60):
#             # break
#     # return new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR
## master function
## use this while running one single optimization; here we used this for EI, UCB, and EPLT optimization campaigns
## policy can be ["MV", 0], ["EPLT", 0], ["EI", 0], or ["UCB", 1. / math.sqrt(2)]
# def master_code(y_target = 1.0, policy = ["UCB", 1. / math.sqrt(2)], num_selection = 1, exp_budget = 61): # to test using the provided dataset
# def master_code(y_target = 1.0, policy = ["EPLT", 0], num_selection = 1, exp_budget = 70): # if EPLT is used
# def master_code(y_target = 1.0, policy = ["UCB", 1. / math.sqrt(2)], num_selection = 1, exp_budget = 70): # if UCB is used
def master_code(y_target = 1.0, policy = ["EI", 0], num_selection = 1, exp_budget = 70): # if EI is used
    loc = input("Please enter the path for the directory that includes files: ")
    print("------------------------------------------------------------------------------------------------------------")
    num_files_old = 0
    while True:
        num_ABS, num_PL = count_files(loc)
        if (num_ABS == num_PL):
            num_files = num_PL
        if (num_files > num_files_old + num_selection - 1) and (num_files < exp_budget):
            print("New run starts now:")
            num_files_old = num_files
            WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR = read_files(loc)
            y = spectra_extract_PL(WL_PL, DR_PL, PL_all, PL_names)
            new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure = exp_sel(y, FR, y_target, policy, num_selection)
            new_FR, updated_FR = x_dim(new_x, FR, loc)
        elif (num_files >= exp_budget):
            print("The experimental budget is reached!")
            break
        ## use this for the test run using the provided dataset 
        # if (num_files_old == 60):
            # break
    # return new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR

In [ ]:
# new_x, new_y, solution_all, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR = master_code()
master_code()